<a href="https://colab.research.google.com/github/totvslabs/carol-notebooks/blob/main/notebooks/AccessAllTenants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Grant Access to All Tenants Within an Organization
`This notebook will grant you access to all Tenants within an Organization.`
`The script will ask for the following json on execution:`

```python
{
    "authentication_config" : {
        "username": "username@totvs.com.br",
        "password": "password",
        "organization": "YourOrganization",
    },
    "script_config" : {
        "role": "userRole",
        "emails": [
          "username@totvs.com.br",
          "username@totvs.com.br",
          "username@totvs.com.br"
        ]
    }
}
```
`You need ORGANIZATION ADMIN permissions to run this script.`

#### REQUIREMENTS
`These are the packages the script needs before execution.`

In [42]:
%%capture
!pip install --quiet pycarol
from pycarol import PwdAuth, Carol
import json

#### CAROL LOGIN FUNCTIONS
`These are the functions made to login into Carol. They will be the same for all notebooks (ideally) and will use pyCarol.` 

[pyCarol reference](https://github.com/totvslabs/pyCarol)

In [43]:
def carol_connect(username, password, organization):
    print(f"Connecting to Carol org {organization}... ", end="\n")

    return Carol(auth=PwdAuth(username, password), org_level=True, organization=organization)

#### SCRIPT FUNCTIONS
`If the script requires more functions to execute, they will be here.`

#### CONFIGURATION FILE
`Now you will need to upload the configuration file with the format given above.`

In [ ]:
try:
    from google.colab import files
    config_file = files.upload()
    config_json = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
    config_json_print = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
except:
    with open('./carol.json') as config_file:
        config_json = json.loads(config_file.read())
        config_json_print = json.loads(config_file.read())
    config_file.close()
finally:
    del config_json_print['authentication_config']['password']
    print(json.dumps(config_json_print, indent=2))

#### SCRIPT EXECUTION
`The main execution of the script will happen here.`

In [ ]:
Carol = carol_connect(
    config_json['authentication_config']['username'], 
    config_json['authentication_config']['password'], 
    config_json['authentication_config']['organization'])

# Add user into the tenants
org_id = Carol._current_org()['mdmId']
all_tenants = Carol.call_api(f'v1/organizations/{org_id}/allTenants', 'GET')
for tenant in all_tenants:
    role = config_json['script_config']['role']
    emails = config_json['script_config']['emails']
    for email in emails:
        tenant_name = tenant['mdmName']
        tenant_id = tenant['mdmId']
        data = {tenant_name:[{"mdmUserLogin":email,"mdmRoleNames":[role]}]}
        q = {
        "mustList": [
            {
            "mdmFilterType": "TERM_FILTER",
            "mdmKey": "mdmUserLogin.raw",
            "mdmValue": email
            }
        ],
        "shouldList": [
        ]
        }
        extra_headers = {'overriddentenantid': tenant_id} 
        params = dict(   
            indexType='CONFIG', 
            scrollable=False,
            pageSize=-1,
        )
        result = Carol.call_api("v1/users/filterUsers", 'POST', data = q,  params=params, extra_headers=extra_headers )
        if len(result['hits'])==0:
            print(f'adding {email} to {tenant_name}')
            Carol.call_api('v1/organizations/assignUsersToEnvironments', 'POST', data = data )